In [1]:
#Author: Aman Bhardwaj
#Entry No.: SIY20197580
#Date: 20-Jan-2020
#Cryptanalysis for Hill Cipher

import numpy as np
import math
import os
import random

#source: https://en.wikipedia.org/wiki/Letter_frequency
dictFreqAlphabets = {"e":0,"t":1,"a":2,"o":3,"i":4,"n":6,"s":7,"h":8,"r":9,"d":10}
#print(listFreqAlphabets)

#source: https://blogs.sas.com/content/iml/2014/09/26/bigrams.html
dictFreqDigrams = {"th":0,"he":1,"in":2,"er":3,"an":4,"re":5,"on":6,"at":7,"en":8,"nd":9,"ti":10,"es":11,"or":12,
                    "te":13,"of":14,"ed":15,"is":16,"it":17,"al":18,"ar":19,"st":20}
#print(listFreqDigraphs)

#source: https://en.wikipedia.org/wiki/Trigram
dictFreqTrigrams = {"the":0,"and":1,"tha":2,"ent":3,"ent":4,"ing":5,"ion":6,"tio":7,"for":8,"nde":9,"has":10}
#print(listFreqTrigraphs)

#form dictionary of chars a-z and map them to 0-25 numbers
def charsDictGen():    
    charsDict = {"a":None}
    init = 'a'
    for i in range(0,26):
        charsDict[init] = i
        init = chr(ord(init) + 1)
    return charsDict

#form dictionary of 0-25 numbers and map them to chars a-z 
def numbersDictGen():    
    numsDict = {0:None}
    init = 'a'
    for i in range(0,26):
        numsDict[i] = init
        init = chr(ord(init) + 1)
    return numsDict

#Key Matrix Generator
def keyGen(key):
    if key:
        key = key.split()
        key = list(map(int, key)) 
        n = int(math.sqrt(len(key)))
        keyMat = np.array(key).reshape(n,n)
        #print(keyMat)

    return keyMat

#calculate modulo inverse of key matrix
def calcModInverse(mat):    
    det = np.linalg.det(mat)
    #print(round(det))
    cofactorMat = np.linalg.inv(mat) * det
    #cofactorMat = cofactorMat.astype(int)
    detMod = calcMultiplicativeInverse(round(det))
    invMat = cofactorMat * detMod
    invMat = np.mod(invMat, 26)

    #print(invMat)
    #print("\n Cofactor Matrix Transpose: \n", cofactorMat, "\n", "\n", invMat, "\n", det, "\n", detMod)
    return invMat.round()
    

#multiplicative inverse of determent mod(26)
def calcMultiplicativeInverse(determinant):
    multi_inverse = -1
    for i in range(26):
        inv = determinant * i
        if inv % 26 == 1:
            multi_inverse = i
            break
    return multi_inverse

#check if key is nxn or not
def isPerfectSquare(k):
    k = k.split()
    k = list(map(int, k))
    sr = math.sqrt(len(k))    
    # If square root is an integer 
    return ((sr - math.floor(sr)) == 0) 

def isInvertible(mat):
    det = np.linalg.det(mat)
    if (int(round(det))!= 0) and (math.gcd(int(round(det)), 26)== 1):
        return True
    else:
        return False
    
def cleanText(text):    
    return text.replace(" ","")

#calculate n-grams and their frequencies
#text #n :- n-grams 
def calcNgramFreq(text, n):
    ngramFreqDict = {}
    for i in range(len(text)-n+1):
        if text[i:(i+n)] in ngramFreqDict:
            ngramFreqDict[text[i:(i+n)]] += 1
        else:
            ngramFreqDict[text[i:(i+n)]] = 1    
    ngramFreqDict = {k: v for k, v in sorted(ngramFreqDict.items(), key=lambda item: item[1], reverse=True)}
    return ngramFreqDict

#Get top t :- top t values
def topCGrams(ngramDict, t):
    ngramDict = {k: v for k, v in sorted(ngramDict.items(), key=lambda item: item[1], reverse=True)[:t]}
    return ngramDict

#Get top t :- top t values
def topPGrams(ngramDict, t):
    ngramDict = {k: v for k, v in sorted(ngramDict.items(), key=lambda item: item[1])[:t]}
    return ngramDict

#create matrix of dimensions nxn from ngrams dictionary
def genNGramMatrix(gramDict, n):
    count=0
    gMat = np.zeros((n, n), dtype=int)
    charsKeyDict = charsDictGen()
    keySplit =[]
    for key in gramDict:
        if count < n*n:
            keySplit.append(list(key))
            count += n
        else:
            break
    keyMat = np.array(keySplit)
    for i in range(n):
        for j in range(n):
            gMat[i][j] = charsKeyDict[keyMat[i][j]]
    return gMat.T

def convertDictToList(dic):
    listGram = []
    for value in dic:
        listGram.append(value)
    return listGram

def genNGMatrix(gramList, n):
    filterGram = []    
    gMat = np.zeros((n, n), dtype=int)    
    charsKeyDict = charsDictGen()
    for i in range(n):
        rnd = random.choice(gramList)
        for ch in rnd:
            filterGram.append(ch)
    count = 0
    for m in range(n):
        for l in range(n):
            gMat[m][l] = charsKeyDict[filterGram[count]]
            count += 1
    return gMat.T

#calculate index of coincidence for given text
def calcIcForText(text):
    textFreqDict = calcNgramFreq(text, 1)
    sigmaFreq = 0
    #print(textFreqDict)
    for ngram in textFreqDict:
        sigmaFreq += textFreqDict[ngram]*(textFreqDict[ngram]-1)
    ic = sigmaFreq / (len(text)*(len(text)-1))
    return ic

#create matrix of plain text of such that it creates linear combinations of vectors of size n (nxn is key size)
def createMatrixOfText(text, n):
    charDict = charsDictGen()
    rem = len(text)%n
    
    if rem != 0:
        xToAppend = n - rem
        
        for i in range(xToAppend):
            text += "z"
    
    quotient = int(len(text)/n)
    textMat = np.zeros((quotient, n), dtype=int)
    
    count = 0
    for m in range(quotient):
        for p in range(n):
            if text[count] != ' ':
                textMat[m][p] = charDict[text[count].lower()]
                count += 1
    #print(textMat.transpose())
    return textMat.transpose()
def matModM(mat):
    for row in mat:
        for elem in row:
            elem = elem%26
    return mat        
#Decrypt the cipher text for a given key using Hill Cipher Algorithm
def hill_decrypt(cText, dKey):
    matCText = createMatrixOfText(cText.replace(' ',''),len(dKey[0]))
    #print(matCText)
    matPText = np.zeros((matCText.shape), dtype = int)
    dKey = calcModInverse(dKey)
    #print("dKey : {}", dKey)
    for i in range(len(matCText[0])):
        matPText[:,i] = dKey.dot(matCText[:,i])

    matPText = matPText.transpose()
    pText = ""
    numToChar = numbersDictGen()
    for row in matPText:
        for element in row:
            element = element%26
            pText += numToChar[element]
    #pText = reInsertSpaces(cText, pText)
    #print("Decrypted Text: \n", pText)
    return pText
    
#crack hill cipher
def letTheHackBegin(cText, kLen):
    cTextGrams = calcNgramFreq(cText, kLen)
    topCGram = topCGrams(cTextGrams, 5)
    listTopCGram = convertDictToList(topCGram)
    pTextMat = []
    topPGram = {}
    listTopPGram = []
    if kLen == 1:
        topPGram = topPGrams(dictFreqAlphabets, 10)
        listTopPGram = convertDictToList(topPGram)
    elif kLen == 2:
        topPGram = topPGrams(dictFreqDigrams, 5)
        listTopPGram = convertDictToList(topPGram)
    elif kLen == 3:
        topPGram = topPGrams(dictFreqTrigrams, 5)
        listTopPGram = convertDictToList(topPGram)
    icVal = 0
    for i in range(5000):
        pMat = genNGMatrix(listTopPGram, kLen)
        cMat = genNGMatrix(listTopCGram, kLen)
        #print(pMat, cMat)
        if isInvertible(pMat):
            pMatInv = calcModInverse(pMat)               
            tempKey = cMat.dot(pMatInv)
            tempKey = np.mod(tempKey, 26)
            #print(tempKey)
            #print(isInvertible(tempKey))
            if isInvertible(tempKey):
                tempKey = tempKey.round()
                decryptedText = hill_decrypt(cText, tempKey)
                icVal = calcIcForText(decryptedText)
                icVal = round(icVal, 4)
                if(icVal > 0.062 and icVal < 0.069):
                    print("\n Deciphered Text:\n", decryptedText)
                    print("\n Number of Iterations", i)
                    print("\n IC value=",icVal)                    
                    print("\n Cracked Key =\n", tempKey)
                    break
    
    if(icVal == 0 or (icVal < 0.062 or icVal > 0.069)):
        print("ALERT:\n Could not find the key combination in this try \n or Key size is not what you thought \n or Encryption key might not be invertible.\n Try again")

def readEncryptedFile():
    try:
        fileName = input("Enter the name of cipher file to decrypt: (Eg. encrypted_text.txt)\n")
        #fileName = "encrypted_text.txt"
        cipherText = ""
        with open(fileName,'r') as cipherFile:
            cipherText = cipherFile.read().replace('\n','')
        return cipherText.lower()
    except:
        print("No such file found in directory. Please check")
        return
    
if __name__ == "__main__":
    #print(os.listdir(os.getcwd()))
    encryptedText = readEncryptedFile()
    if encryptedText is not None:
        if encryptedText:
            cText = cleanText(encryptedText)
            keyLen = input("Enter Key Length(Eg. for 2x2 enter 4)")
            sr = math.sqrt(int(keyLen))
            if ((sr - math.floor(sr)) == 0):
                keyLen = math.sqrt(int(keyLen))
                calcIcForText(cText)
                print("Wait. This may take some time...!")
                letTheHackBegin(cText, round(keyLen))
            else:
                print("ALERT : Key is not nxn. It should be a square")
        else:
            print("ALERT: File is empty.")

Enter the name of cipher file to decrypt: (Eg. encrypted_text.txt)
encrypted_text.txt
Enter Key Length(Eg. for 2x2 enter 4)4
Wait. This may take some time...!

 Deciphered Text:
 speakingofdreamsinafigurativesensethenslowlytalkingaboutitsimpactintheliteralsensethewriterkeepsstressingononeveryimportantpointthatdreamshaveanimportantroletoplayinamericanpoliticsafterallaliberalsocietyisformedonthebasisofonesimaginationtheseimaginationsarearesultofonesfreethoughtswhichcanberelatedtodreamsasdreamingprovidesapictureoftherealworlduncoveringthingswhichotherwisemightnothavebeenpondereduponduetonarrowedandlimitedfreedomofimaginingitmayseemrecklesstoconsiderthepossibilityofturningtodreamstoworkthroughthepoliticalconditionstodaybutignoringthemaltogethermightevenbemorerecklesssaystheauthorz

 Number of Iterations 34

 IC value= 0.0667

 Cracked Key =
 [[22.  3.]
 [ 9.  6.]]
